Renaming the files

In [ ]:
import os

# Path to the folder containing the Excel files
folder_path = r'C:\Users\Satish\Downloads\Survey-responses'

# Get a list of all Excel files in the folder
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Rename each file
for i, file_name in enumerate(excel_files, start=1):
    new_name = f'survey-{i}.xlsx'
    old_file = os.path.join(folder_path, file_name)
    new_file = os.path.join(folder_path, new_name)
    os.rename(old_file, new_file)
    print(f'Renamed: {file_name} -> {new_name}')


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Satish\\Downloads\\Survey-responses\\Compact_Restructured_Survey2_Output.xlsx' -> 'C:\\Users\\Satish\\Downloads\\Survey-responses\\survey-1.xlsx'

Combining the different excel files into one

In [ ]:
import pandas as pd
import glob
import os

# Function to process a single Excel file
def process_excel_file(file_path):
    # Load Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # Check if 'Responses' sheet exists
    if 'Responses' in excel_file.sheet_names:
        # Process Sheet 1: Survey Responses
        df_survey = pd.read_excel(excel_file, 'Responses', header=0)
        
        # Determine Survey Type based on the 2nd question
        survey_type = None
        if df_survey.columns[1] == "What is your name?":
            survey_type = "Survey1"
        else:
            survey_type = "Survey2"
        
        # Load other sheets (if needed, currently not processed differently based on survey type)
        df_timings = pd.read_excel(excel_file, 'Time Taken', header=0)
        df_coordinates = pd.read_excel(excel_file, 'Mouse Movement', header=0)
        
        # Return the dataframes and survey type
        return df_survey, df_timings, df_coordinates, survey_type
    else:
        print(f"Worksheet 'Responses' not found in file: {file_path}")
        return None, None, None, None

# Function to combine all Excel files into two separate Excel files based on Survey Type
def combine_all_excel_files(directory_path):
    survey1_responses = []
    survey1_timings = []
    survey1_coordinates = []
    survey2_responses = []
    survey2_timings = []
    survey2_coordinates = []
    
    # Find all Excel files in the directory
    for file_path in glob.glob(directory_path + "/*.xlsx"):
        df_survey, df_timings, df_coordinates, survey_type = process_excel_file(file_path)
        
        # Append to the respective list if the worksheet was found
        if df_survey is not None:
            if survey_type == "Survey1":
                survey1_responses.append(df_survey)
                survey1_timings.append(df_timings)
                survey1_coordinates.append(df_coordinates)
            elif survey_type == "Survey2":
                survey2_responses.append(df_survey)
                survey2_timings.append(df_timings)
                survey2_coordinates.append(df_coordinates)
    
    # Combine all responses into a single dataframe for each survey type
    combined_survey1_responses = pd.concat(survey1_responses, ignore_index=True)
    combined_survey1_timings = pd.concat(survey1_timings, ignore_index=True)
    combined_survey1_coordinates = pd.concat(survey1_coordinates, ignore_index=True)
    combined_survey2_responses = pd.concat(survey2_responses, ignore_index=True)
    combined_survey2_timings = pd.concat(survey2_timings, ignore_index=True)
    combined_survey2_coordinates = pd.concat(survey2_coordinates, ignore_index=True)
    
    # Transform the Mouse Movement data to have time as columns and x, y coordinates as rows
    if 'Time' in combined_survey1_coordinates.columns and 'X' in combined_survey1_coordinates.columns and 'Y' in combined_survey1_coordinates.columns:
        combined_survey1_coordinates['Coordinates'] = combined_survey1_coordinates.apply(lambda row: f"{row['X']},{row['Y']}", axis=1)
        combined_survey1_coordinates = combined_survey1_coordinates.pivot(index=None, columns='Time', values='Coordinates')
    if 'Time' in combined_survey2_coordinates.columns and 'X' in combined_survey2_coordinates.columns and 'Y' in combined_survey2_coordinates.columns:
        combined_survey2_coordinates['Coordinates'] = combined_survey2_coordinates.apply(lambda row: f"{row['X']},{row['Y']}", axis=1)
        combined_survey2_coordinates = combined_survey2_coordinates.pivot(index=None, columns='Time', values='Coordinates')
    
    # Save the combined dataframes to new Excel files
    with pd.ExcelWriter("New_Combined_Survey1_Output.xlsx") as writer:
        combined_survey1_responses.to_excel(writer, sheet_name="Responses", index=False)
        combined_survey1_timings.to_excel(writer, sheet_name="Time Taken", index=False)
        combined_survey1_coordinates.to_excel(writer, sheet_name="Mouse Movement", index=False)
    
    with pd.ExcelWriter("New_Combined_Survey2_Output.xlsx") as writer:
        combined_survey2_responses.to_excel(writer, sheet_name="Responses", index=False)
        combined_survey2_timings.to_excel(writer, sheet_name="Time Taken", index=False)
        combined_survey2_coordinates.to_excel(writer, sheet_name="Mouse Movement", index=False)

# Usage
if __name__ == "__main__":
    directory_path = input("Enter the directory path containing your Excel files: ")
    combine_all_excel_files(directory_path)
    print("All files have been processed and combined into 'New_Combined_Survey1_Output.xlsx' and 'New_Combined_Survey2_Output.xlsx'.")


ValueError: No objects to concatenate

Combining the responses and time takes sheet of the combined excel

In [11]:
import pandas as pd
import json

# Load the Excel file
file_path = r'C:\Users\Satish\Downloads\Survey-responses\New_Combined_Survey2_Output.xlsx'
excel_file = pd.ExcelFile(file_path)

# Check if 'Responses' and 'Time Taken' sheets exist
if 'Responses' in excel_file.sheet_names and 'Time Taken' in excel_file.sheet_names:
    # Load the sheets
    df_responses = pd.read_excel(excel_file, 'Responses', header=0)
    df_timings = pd.read_excel(excel_file, 'Time Taken', header=0)
    
    # Combine the data into JSON format
    combined_data = []
    for i, row in df_responses.iterrows():
        combined_row = {}
        for col in df_responses.columns:
            # Match the corresponding column in the 'Time Taken' sheet
            timing_col = f"Q{df_responses.columns.get_loc(col) + 1} (seconds)"
            combined_row[col] = json.dumps({
                "responses": row[col],
                "time taken": df_timings.at[i, timing_col] if timing_col in df_timings.columns else None
            })
        combined_data.append(combined_row)
    
    # Convert to DataFrame
    combined_df = pd.DataFrame(combined_data)
    
    # Save the combined DataFrame to a new Excel file
    combined_df.to_excel('Combined_Survey_Output.xlsx', index=False)
    print("Combined data has been saved to 'Combined_Survey_Output.xlsx'")
else:
    print("Worksheet 'Responses' or 'Time Taken' not found in the file.")


Combined data has been saved to 'Combined_Survey_Output.xlsx'


In [30]:
import pandas as pd
import json
import glob
import os

# Function to process a single Excel file
def process_excel_file(file_path):
    # Load Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # Check if 'Mouse Movement' sheet exists and if B1 contains the specified header
    if 'Mouse Movement' in excel_file.sheet_names and 'Responses' in excel_file.sheet_names:
        df_responses = pd.read_excel(excel_file, 'Responses', header=0)
        if df_responses.columns[1] == "What do you think is the most exciting application of AI in today's world?":
            df_mouse_movement = pd.read_excel(excel_file, 'Mouse Movement', header=0)
            
            # Process the data into JSON format
            movement_list = []
            for i, row in df_mouse_movement.iterrows():
                for col in df_mouse_movement.columns:
                    if pd.notna(row[col]):
                        value = str(row[col])
                        if ',' in value:
                            time, coordinates = value.split(',')
                            movement_list.append({
                                "time": time.strip(),
                                "coordinates": coordinates.strip()
                            })
            
            # Convert the movement list to JSON format
            movement_json = json.dumps(movement_list)
            
            return movement_json
    else:
        print(f"Worksheet 'Mouse Movement' or 'Responses' not found in file: {file_path}")
        return None

# Function to combine all Excel files into one sheet in JSON format
def combine_all_excel_files(directory_path):
    combined_data = []
    
    # Find all Excel files in the directory
    for file_path in glob.glob(directory_path + "/*.xlsx"):
        movement_json = process_excel_file(file_path)
        
        # Append to the combined data if the worksheet was found
        if movement_json is not None:
            combined_data.append({
                "mouse_movement": movement_json
            })
    
    # Convert the combined data to a DataFrame
    final_combined_df = pd.DataFrame(combined_data)
    
    # Save the combined DataFrame to a new Excel file
    final_combined_df.to_excel('Mouse_Movement_Output.xlsx', index=False)
    print("Mouse movement data has been saved to 'Mouse_Movement_Output.xlsx'")

# Usage
if __name__ == "__main__":
    directory_path = r'C:\Users\Satish\Downloads\Survey-responses'
    combine_all_excel_files(directory_path)
    print("All files have been processed and combined into 'Mouse_Movement_Output.xlsx'.")


Worksheet 'Mouse Movement' or 'Responses' not found in file: C:\Users\Satish\Downloads\Survey-responses\Mouse_Movement_Output.xlsx
Mouse movement data has been saved to 'Mouse_Movement_Output.xlsx'
All files have been processed and combined into 'Mouse_Movement_Output.xlsx'.


Combinging the mouse movement

In [43]:
import pandas as pd
import json
import glob
import os

# Function to process a single Excel file
def process_excel_file(file_path):
    # Load Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # Check if 'Mouse Movement' sheet exists and if B1 contains the specified header
    if 'Mouse Movement' in excel_file.sheet_names and 'Responses' in excel_file.sheet_names:
        df_responses = pd.read_excel(excel_file, 'Responses', header=0)
        if df_responses.columns[1] == "What do you think is the most exciting application of AI in today's world?":
            df_mouse_movement = pd.read_excel(excel_file, 'Mouse Movement', header=0)
            
            # Process the data into JSON format
            movement_list = []
            for _, row in df_mouse_movement.iterrows():
                if pd.notna(row['Elapsed Time (seconds)']) and pd.notna(row['X-Y Coordinate']):
                    time = str(row['Elapsed Time (seconds)']).strip()
                    coordinates = str(row['X-Y Coordinate']).strip()
                    movement_list.append({
                        "time": time,
                        "coordinates": coordinates
                    })
            
            # Convert the movement list to JSON format
            movement_json = json.dumps(movement_list)
            
            return movement_json
    else:
        print(f"Worksheet 'Mouse Movement' or 'Responses' not found in file: {file_path}")
        return None

# Function to combine all Excel files into one sheet in JSON format
def combine_all_excel_files(directory_path):
    combined_data = []
    
    # Find all Excel files in the directory
    for file_path in glob.glob(directory_path + "/*.xlsx"):
        movement_json = process_excel_file(file_path)
        
        # Append to the combined data if the worksheet was found
        if movement_json is not None:
            combined_data.append({
                "mouse_movement": movement_json
            })
    
    # Convert the combined data to a DataFrame
    final_combined_df = pd.DataFrame(combined_data)
    
    # Save the combined DataFrame to a new Excel file
    final_combined_df.to_excel('Mouse_Movement_Output.xlsx', index=False)
    print("Mouse movement data has been saved to 'Mouse_Movement_Output.xlsx'")

# Usage
if __name__ == "__main__":
    directory_path = r'C:\Users\Satish\Downloads\Survey-responses'
    combine_all_excel_files(directory_path)
    print("All files have been processed and combined into 'Mouse_Movement_Output.xlsx'.")


Worksheet 'Mouse Movement' or 'Responses' not found in file: C:\Users\Satish\Downloads\Survey-responses\Mouse_Movement_Output.xlsx
Mouse movement data has been saved to 'Mouse_Movement_Output.xlsx'
All files have been processed and combined into 'Mouse_Movement_Output.xlsx'.


Converting mouse mouvement to avg velocity 

In [58]:
import pandas as pd
import json
import numpy as np

# Load the dataset
file_path = r'C:\Users\Satish\Downloads\Survey-responses\Mouse_Movement_Output.xlsx'
df = pd.read_excel(file_path)

# Function to calculate the number of pauses (stationary positions)
def calculate_pauses(row, threshold=5, stationary_time=2):
    try:
        # Parse the JSON string from the mouse_movement column
        movements = json.loads(row)
        
        # Check if the data is valid (should be a list of objects)
        if not isinstance(movements, list) or len(movements) < 2:
            return 0
        
        # Initialize variables to count pauses
        pauses = 0
        consecutive_stationary_count = 0
        
        # Loop through consecutive points to detect pauses
        for i in range(1, len(movements)):
            t1 = float(movements[i-1]['time'])
            t2 = float(movements[i]['time'])
            x1, y1 = map(int, movements[i-1]['coordinates'].split(','))
            x2, y2 = map(int, movements[i]['coordinates'].split(','))
            
            # Calculate the distance between the points
            distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            
            # If the distance is below the threshold, it's considered a stationary point
            if distance < threshold:
                consecutive_stationary_count += 1
            else:
                if consecutive_stationary_count >= stationary_time:
                    pauses += 1  # Considered a pause
                consecutive_stationary_count = 0
        
        # Check for a pause at the end of the movements
        if consecutive_stationary_count >= stationary_time:
            pauses += 1
        
        return pauses
    
    except json.JSONDecodeError as e:
        # In case of a JSON decoding error, log the issue
        print(f"JSONDecodeError: {e} in row {row}")
        return 0
    except Exception as e:
        # Handle any other exceptions (e.g., invalid data)
        print(f"Error: {e} in row {row}")
        return 0

# Step 2: Apply the calculation to each row in the dataframe
df['num_pauses'] = df['mouse_movement'].apply(calculate_pauses)

# Step 3: Save the results to a new Excel file
output_path = r'C:\Users\Satish\Downloads\Survey-responses\Mouse_Movement_Output_with_num_pauses.xlsx'
df.to_excel(output_path, index=False)

print(f'File saved to {output_path}')


JSONDecodeError: Unterminated string starting at: line 1 column 32761 (char 32760) in row [{"time": "0.02", "coordinates": "136,25"}, {"time": "0.03", "coordinates": "154,48"}, {"time": "0.04", "coordinates": "182,84"}, {"time": "0.06", "coordinates": "205,115"}, {"time": "0.08", "coordinates": "234,156"}, {"time": "0.09", "coordinates": "241,168"}, {"time": "0.12", "coordinates": "260,192"}, {"time": "0.13", "coordinates": "260,193"}, {"time": "0.14", "coordinates": "260,194"}, {"time": "0.16", "coordinates": "256,194"}, {"time": "0.17", "coordinates": "249,194"}, {"time": "0.19", "coordinates": "238,192"}, {"time": "0.21", "coordinates": "229,192"}, {"time": "0.66", "coordinates": "236,195"}, {"time": "0.66", "coordinates": "246,199"}, {"time": "0.68", "coordinates": "257,204"}, {"time": "0.69", "coordinates": "289,222"}, {"time": "0.71", "coordinates": "318,237"}, {"time": "0.72", "coordinates": "325,240"}, {"time": "0.74", "coordinates": "343,248"}, {"time": "0.76", "coordinates": 

Code for data transformation

In [90]:
import pandas as pd
import json
from collections import Counter
data = {
    "Category": ["Genuine", "Rush", "Genuine", "Genuine", "Genuine","Rush", "Genuine", "Rush", "Rush", "Genuine", "Rush", "Genuine", "Rush", "Rush"],
    "SUB(Q1)": [
        '{"response_length": 39, "time taken": 6.47}', '{"response_length": 12, "time taken": 13.81}', '{"response_length": 7, "time taken": 13.34}', '{"response_length": 7, "time taken": 13.34}', '{"response_length": 30, "time taken": 9.95}', '{"response_length": 1, "time taken": 7.09}', '{"response_length": 26, "time taken": 7.38}', '{"response_length": 2, "time taken": 6.5}', '{"response_length": 7, "time taken": 17.93}', '{"response_length": 27, "time taken": 9.67}', '{"response_length": 11, "time taken": 6.04}'
        ,  '{"response_length": 4, "time taken": 5.29}', '{"response_length": 4, "time taken": 2.54}', '{"response_length": 0, "time taken": 6.9}'
        ],
    "MCQ(Q2)": [
        '{"responses": "b", "time taken": 19.23}', '{"responses": "a", "time taken": 20.47}', '{"responses": "a", "time taken": 24.26}', '{"responses": "a", "time taken": 24.26}', '{"responses": "c", "time taken": 21.93}', '{"responses": "a", "time taken": 8.47}', '{"responses": "e", "time taken": 15.62}', '{"responses": "a", "time taken": 7.94}', '{"responses": "e", "time taken": 22.71}', '{"responses": "b", "time taken": 26.23}', '{"responses": "b", "time taken": 10.62}'
    ,  '{"responses": "a", "time taken": 7.65}', '{"responses": "d", "time taken": 5.1}', '{"responses": "e", "time taken": 10.79}'
    ],
    "MMCQ(Q1)": [
        '{"responses": "b, c, a, d, e", "time taken": 24.93}', '{"responses": "a, c", "time taken": 31.55}', '{"responses": "d, c", "time taken": 37.27}', '{"responses": "d, c", "time taken": 37.27}', '{"responses": "a, d", "time taken": 25.18}'
    , '{"responses": "b, d", "time taken": 9.52}', '{"responses": "e, b", "time taken": 17.42}', '{"responses": "c, b", "time taken": 10.31}', '{"responses": "d, e", "time taken": 28.03}', '{"responses": "b, c", "time taken": 33.16}', '{"responses": "a, c", "time taken": 12.22}'
    ,  '{"responses": "a", "time taken": 8.7}', '{"responses": "c", "time taken": 7.24}', '{"responses": "d", "time taken": 12.17}'
    ],
    "SUB(Q2)": [
        '{"response_length": 87, "time taken": 33.51}', '{"response_length_length": 2, "time taken": 35.69}', '{"response_length_length": 48, "time taken": 461.29}', '{"response_length_length": 48, "time taken": 461.29}', '{"response_length": 39, "time taken": 32.61}'
    ,  '{"response_length": 1, "time taken": 13.12}', '{"response_length": 39, "time taken": 22.73}', '{"response_length": 2, "time taken": 14.28}', '{"response_length": 0, "time taken": 0}', '{"response_length": 41, "time taken": 41.34}', '{"response_length": 6, "time taken": 16.03}'
    , '{"response_length": 1, "time taken": 11.59}', '{"response_length": 5, "time taken": 8.33}', '{"response_length": 0, "time taken": 14.04}'
    ],
    "MCQ(Q3)": [
        '{"responses": "c", "time taken": 75.48}', '{"responses": "d", "time taken": 40.93}', '{"responses": "a", "time taken": 55.14}', '{"responses": "a", "time taken": 55.14}', '{"responses": "e", "time taken": 53.88}'
    ,  '{"responses": "", "time taken": 0}', '{"responses": "a", "time taken": 34.18}', '{"responses": "c", "time taken": 16.1}', '{"responses": "a", "time taken": 32.98}', '{"responses": "d", "time taken": 61.07}', '{"responses": "b", "time taken": 19.31}'
    ,  '{"responses": "a", "time taken": 12.51}', '{"responses": "d", "time taken": 10.35}', '{"responses": "c", "time taken": 15.35}'
    ],
    "MMCQ(Q2)": [
        '{"responses": "a", "time taken": 76.6}', '{"responses": "a, e", "time taken": 44.96}', '{"responses": "a, b, e, d, c", "time taken": 58.03}', '{"responses": "a, b, e, d, c", "time taken": 58.03}', '{"responses": "a", "time taken": 59.96}'
    ,  '{"responses": "a", "time taken": 15.29}', '{"responses": "e, b", "time taken": 37.27}', '{"responses": "d", "time taken": 19.34}', '{"responses": "e", "time taken": 36.97}', '{"responses": "a, e", "time taken": 73.12}', '{"responses": "e, d", "time taken": 21.03}'
    ,  '{"responses": "a", "time taken": 13.97}', '{"responses": "b", "time taken": 11.78}', '{"responses": "d", "time taken": 16.83}'
    ],
    "SUB(Q3)": [
        '{"response_length": 39, "time taken": 84.6}', '{"response_length": 44, "time taken": 52.35}', '{"response_length": 39, "time taken": 539.54}', '{"response_length": 39, "time taken": 539.54}', '{"response_length": 47, "time taken": 64.22}'
    ,  '{"response_length": 1, "time taken": 18.52}', '{"response_length": 39, "time taken": 42.92}', '{"response_length": 1, "time taken": 22.59}', '{"response_length": 9, "time taken": 43.16}', '{"response_length": 30, "time taken": 82.87}', '{"response_length": 16, "time taken": 27.97}' 
    ,         '{"response_length": 1, "time taken": 17.43}', '{"response_length": 4, "time taken": 13.08}', '{"response_length": 0, "time taken": 18.12}'
    ],
    "MCQ(Q4)": [
        '{"responses": "c", "time taken": 110.52}', '{"responses": "d", "time taken": 67.39}', '{"responses": "b", "time taken": 731.11}', '{"responses": "b", "time taken": 731.11}', '{"responses": "d", "time taken": 77.06}'
    ,  '{"responses": "a", "time taken": 19.63}', '{"responses": "a", "time taken": 51.76}', '{"responses": "d", "time taken": 24.47}', '{"responses": "c", "time taken": 45.91}', '{"responses": "b", "time taken": 97.63}', '{"responses": "d", "time taken": 34.77}'
    ,         '{"responses": "a", "time taken": 18.67}', '{"responses": "e", "time taken": 15.08}', '{"responses": "e", "time taken": 19.49}'
    ],
    "SUB(Q4)": [
        '{"response_length": 33, "time taken": 114.99}', '{"response_length": 13, "time taken": 72.99}', '{"response_length": 44, "time taken": 450.9}', '{"response_length": 44, "time taken": 450.9}', '{"response_length": 44, "time taken": 82.78}'
    , '{"response_length": 3, "time taken": 22.5}', '{"response_length": 25, "time taken": 57.71}', '{"response_length": 0, "time taken": 0}', '{"response_length": 0, "time taken": 0}', '{"response_length": 39, "time taken": 100.82}', '{"response_length": 11, "time taken": 40.85}'
    ,  '{"response_length": 0, "time taken": 0}', '{"response_length": 5, "time taken": 16.11}', '{"response_length": 0, "time taken": 21.76}'
    ],
    "MCQ(Q5)": [
        '{"responses": "b", "time taken": 138.23}', '{"responses": "c", "time taken": 87.49}', '{"responses": "e", "time taken": 571.59}', '{"responses": "e", "time taken": 571.59}', '{"responses": "d", "time taken": 97.5}'
    ,   '{"responses": "a", "time taken": 24.13}', '{"responses": "d", "time taken": 67.69}', '{"responses": "a", "time taken": 26.59}', '{"responses": "d", "time taken": 49.84}', '{"responses": "d", "time taken": 116.61}', '{"responses": "a", "time taken": 47.07}'
    ,  '{"responses": "a", "time taken": 20.95}', '{"responses": "b", "time taken": 20.59}', '{"responses": "c", "time taken": 23.13}'
    ],
    "MMCQ(Q3)": [
        '{"responses": "c", "time taken": 144.2}', '{"responses": "a", "time taken": 102.93}', '{"responses": "d, b", "time taken": 120.23}', '{"responses": "d, b", "time taken": 120.23}', '{"responses": "e, b", "time taken": 101.95}'
    ,   '{"responses": "a, b, c, e", "time taken": 24.91}', '{"responses": "a, c", "time taken": 72.45}', '{"responses": "a, b", "time taken": 27.77}', '{"responses": "d", "time taken": 52.12}', '{"responses": "b", "time taken": 127.4}', '{"responses": "d", "time taken": 48.6}'
    ,  '{"responses": "a, d, e", "time taken": 21.89}', '{"responses": "d", "time taken": 21.69}', '{"responses": "c", "time taken": 24.58}'
    ],
    "MCQ(Q6)": [
        '{"responses": "c", "time taken": 146.53}', '{"responses": "b", "time taken": 108.3}', '{"responses": "a", "time taken": 112.54}', '{"responses": "a", "time taken": 112.54}', '{"responses": "d", "time taken": 108.05}'
    ,  '{"responses": "a", "time taken": 28.21}', '{"responses": "e", "time taken": 78.95}', '{"responses": "d", "time taken": 30.09}', '{"responses": "c", "time taken": 56.26}', '{"responses": "d", "time taken": 136.09}', '{"responses": "e", "time taken": 51.0}'
    ,  '{"responses": "a", "time taken": 24.68}', '{"responses": "a", "time taken": 23.06}', '{"responses": "c", "time taken": 25.57}'
    ],
    "SUB(Q5)": [
        '{"response_length": 47, "time taken": 149.92}', '{"response_length": 0, "time taken": 0}', '{"response_length": 28, "time taken": 625.07}', '{"response_length": 28, "time taken": 625.07}', '{"response_length": 47, "time taken": 112.23}'
    ,  '{"response_length": 0, "time taken": 0}', '{"response_length": 47, "time taken": 83.09}', '{"response_length": 1, "time taken": 32.05}', '{"response_length": 0, "time taken": 0}', '{"response_length": 33, "time taken": 146.27}', '{"response_length": 0, "time taken": 0}'
    ,  '{"response_length": 1, "time taken": 25.11}', '{"response_length": 4, "time taken": 24.15}', '{"response_length": 0, "time taken": 27.28}' 
    ],
    "MMCQ(Q4)": [
        '{"responses": "b, a, c, e, d", "time taken": 162.32}', '{"responses": "a, b", "time taken": 116.35}', '{"responses": "b, c, d", "time taken": 97.53}', '{"responses": "b, c, d", "time taken": 97.53}', '{"responses": "a, d", "time taken": 124.97}'
    ,  '{"responses": "a", "time taken": 32.15}', '{"responses": "a, b, c, e", "time taken": 98.19}', '{"responses": "b", "time taken": 33.62}', '{"responses": "d", "time taken": 60.04}', '{"responses": "c, b", "time taken": 156.74}', '{"responses": "a", "time taken": 56.37}'
    ,  '{"responses": "a", "time taken": 27.39}', '{"responses": "d", "time taken": 26.21}', '{"responses": "e", "time taken": 28.69}'
    ],
    "MCQ(Q7)": [
        '{"responses": "a", "time taken": 168.13}', '{"responses": "a", "time taken": 123.78}', '{"responses": "b", "time taken": 91.2}', '{"responses": "b", "time taken": 91.2}', '{"responses": "d", "time taken": 134.03}'
    ,  '{"responses": "b", "time taken": 36.15}', '{"responses": "c", "time taken": 107.43}', '{"responses": "b", "time taken": 34.88}', '{"responses": "c", "time taken": 63.61}', '{"responses": "a", "time taken": 163.9}', '{"responses": "c", "time taken": 58.1}'
    , '{"responses": "d", "time taken": 31.86}', '{"responses": "a", "time taken": 27.61}', '{"responses": "d", "time taken": 29.83}'
      ],
    "SUB(Q6)": [
        '{"response_length": 2, "time taken": 172.03}', '{"response_length": 24, "time taken": 129.8}', '{"response_length": 57, "time taken": 657.35}', '{"response_length": 57, "time taken": 657.35}', '{"response_length": 31, "time taken": 139.83}'
    ,  '{"responses": "b", "time taken": 36.15}', '{"responses": "c", "time taken": 107.43}', '{"responses": "b", "time taken": 34.88}', '{"responses": "c", "time taken": 63.61}', '{"responses": "a", "time taken": 163.9}', '{"responses": "c", "time taken": 58.1}'
    ,  '{"response_length": 1, "time taken": 30.12}', '{"response_length": 5, "time taken": 28.51}', '{"response_length": 0, "time taken": 31.53}'
    ],
    "MMCQ(Q5)": [
        '{"responses": "b", "time taken": 176.82}', '{"responses": "a, d", "time taken": 143.05}', '{"responses": "e", "time taken": 75.66}', '{"responses": "e", "time taken": 75.66}', '{"responses": "a, d", "time taken": 151.14}'
    ,  '{"responses": "a", "time taken": 40.62}', '{"responses": "a, c", "time taken": 117.88}', '{"responses": "d", "time taken": 42.07}', '{"responses": "d, e", "time taken": 69.92}', '{"responses": "b", "time taken": 184.71}', '{"responses": "d", "time taken": 67.31}'
    ,  '{"responses": "a", "time taken": 33.34}', '{"responses": "c", "time taken": 30.54}', '{"responses": "e", "time taken": 33.8}'
    ],
    "MCQ(Q8)": [
        '{"responses": "c", "time taken": 178.1}', '{"responses": "b", "time taken": 147.87}', '{"responses": "a", "time taken": 71.28}', '{"responses": "a", "time taken": 71.28}', '{"responses": "c", "time taken": 158.16}'
    ,  '{"responses": "b", "time taken": 45.04}', '{"responses": "b", "time taken": 123.19}', '{"responses": "d", "time taken": 43.79}', '{"responses": "d", "time taken": 72.6}', '{"responses": "a", "time taken": 192.23}', '{"responses": "d", "time taken": 70.14}'
    ,  '{"responses": "a", "time taken": 35.43}', '{"responses": "d", "time taken": 33.24}', '{"responses": "e", "time taken": 35.36}'
    ],
    "SUB(Q7)": [
        '{"response_length": 88, "time taken": 184.83}', '{"response_length": 2, "time taken": 160.44}', '{"response_length": 2, "time taken": 717.28}', '{"response_length": 2, "time taken": 717.28}', '{"response_length": 39, "time taken": 161.39}'
    ,  '{"response_length": 10, "time taken": 46.55}', '{"response_length": 31, "time taken": 126.45}', '{"response_length": 4, "time taken": 45.67}', '{"response_length": 2, "time taken": 74.48}', '{"response_length": 22, "time taken": 196.52}', '{"response_length": 9, "time taken": 72.46}'
    ,  '{"response_length": 1, "time taken": 36.28}', '{"response_length": 3, "time taken": 34.93}', '{"response_length": 0, "time taken": 36.73}'
    ],
    "Non-attempt": [0, 1, 0, 0, 0, 2, 0, 1, 3, 0, 1, 1, 0, 0]
}
df = pd.DataFrame(data)
# Function to calculate selection pattern, unique options, most common option, and diversity score
def calc_mcq_features(columns):
    selection_patterns = []
    unique_options_list = []
    most_common_options = []
    diversity_scores = []
    
    for _, row in df.iterrows():
        all_responses = []
        for col in columns:
            try:
                response_data = json.loads(row[col])
                responses = response_data["responses"].split(", ")
                all_responses.extend(responses)
            except KeyError:
                continue
        # Calculate features
        selection_patterns.append(" -> ".join(all_responses))
        unique_options = set(all_responses)
        unique_options_list.append(len(unique_options))
        most_common_option = Counter(all_responses).most_common(1)
        most_common_options.append(most_common_option[0][0] if most_common_option else None)
        diversity_scores.append(len(unique_options) / len(all_responses) if all_responses else 0)
    
    return selection_patterns, unique_options_list, most_common_options, diversity_scores

# MCQ/MMCQ columns
mcq_cols = [col for col in df.columns if col.startswith("MCQ") or col.startswith("MMCQ")]

# Add MCQ features
df["Selection_Pattern"], df["Unique_Options"], df["Most_Common_Option"], df["Diversity_Score"] = calc_mcq_features(mcq_cols)

# Function to calculate total time for MCQ/MMCQ questions
def calc_total_time(columns):
    total_times = []
    for _, row in df.iterrows():
        total_time = 0
        for col in columns:
            try:
                response_data = json.loads(row[col])
                time_taken = response_data["time taken"]
                total_time += time_taken
            except KeyError:
                continue
        total_times.append(total_time)
    return total_times

# Add total_Time_Per_Response column
df["Total_Time_Per_Response"] = calc_total_time(mcq_cols)

# Function to calculate subjective metrics (length and time)
def calc_subjective_metrics(columns):
    total_lengths = []
    total_times = []
    for _, row in df.iterrows():
        total_length = 0
        total_time = 0
        for col in columns:
            try:
                response_data = json.loads(row[col])
                total_length += response_data.get("response_length", 0)
                total_time += response_data.get("time taken", 0)
            except KeyError:
                continue
        total_lengths.append(total_length)
        total_times.append(total_time)
    return total_lengths, total_times

# Subjective question columns
sub_cols = [col for col in df.columns if col.startswith("SUB")]

# Add Total_Sub_Length and Total_Sub_Time columns
df['Total_Sub_Length'], df['Total_Sub_Time'] = calc_subjective_metrics(sub_cols)

# Calculate Avg_Typing_Speed (length per second)
df['Avg_Typing_Speed'] = df['Total_Sub_Length'] / df['Total_Sub_Time']

# Final dataframe
final_columns = ['Category', 'Selection_Pattern', 'Unique_Options', 'Most_Common_Option',
                 'Diversity_Score', 'Total_Time_Per_Response', 'Total_Sub_Length', 
                 'Total_Sub_Time', 'Avg_Typing_Speed', 'Non-attempt']
final_df = df[final_columns]

# Display the final processed DataFrame
print(final_df)

output_file = "processed_data2.xlsx"
final_df.to_excel(output_file, index=False)

print(f"Data has been successfully saved to {output_file}")


   Category                                  Selection_Pattern  \
0   Genuine  b -> b -> c -> a -> d -> e -> c -> a -> c -> b...   
1      Rush  a -> a -> c -> d -> a -> e -> d -> c -> a -> b...   
2   Genuine  a -> d -> c -> a -> a -> b -> e -> d -> c -> b...   
3   Genuine  a -> d -> c -> a -> a -> b -> e -> d -> c -> b...   
4   Genuine  c -> a -> d -> e -> a -> d -> d -> e -> b -> d...   
5      Rush  a -> b -> d ->  -> a -> a -> a -> a -> b -> c ...   
6   Genuine  e -> e -> b -> a -> e -> b -> a -> d -> a -> c...   
7      Rush  a -> c -> b -> c -> d -> d -> a -> a -> b -> d...   
8      Rush  e -> d -> e -> a -> e -> c -> d -> d -> c -> d...   
9   Genuine  b -> b -> c -> d -> a -> e -> b -> d -> b -> d...   
10     Rush  b -> a -> c -> b -> e -> d -> d -> a -> d -> e...   
11  Genuine  a -> a -> a -> a -> a -> a -> a -> d -> e -> a...   
12     Rush  d -> c -> d -> b -> e -> b -> d -> a -> d -> a...   
13     Rush  e -> d -> c -> d -> e -> c -> c -> c -> e -> d...   

    Uniqu

CALLING THE DATASET

In [92]:
import pandas as pd

# Load the dataset from the specified path
file_path = r"C:\Users\Satish\Downloads\Survey-responses\result\adjusted_dataset.csv"
data = pd.read_csv(file_path)

# Preview the dataset
print(data.head())


  Category                                  Selection_Pattern  Unique_Options  \
0  Genuine  b -> b -> c -> a -> d -> e -> c -> a -> c -> b...               5   
1     Rush  a -> a -> c -> d -> a -> e -> d -> c -> a -> b...               5   
2  Genuine  a -> d -> c -> a -> a -> b -> e -> d -> c -> b...               5   
3  Genuine  a -> d -> c -> a -> a -> b -> e -> d -> c -> b...               5   
4  Genuine  c -> a -> d -> e -> a -> d -> d -> e -> b -> d...               5   

  Most_Common_Option  Diversity_Score  Total_Time_Per_Response  \
0                  c           0.2500                     1421   
1                  a           0.3125                     1035   
2                  a           0.2500                     2045   
3                  a           0.2500                     2045   
4                  d           0.3125                     1113   

   Total_Sub_Length  Total_Sub_Time  Avg_Typing_Speed  Non-attempt  
0            746.35        0.448851          0.

preprocessing for GBT

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Define features and target
X = data.drop('Category', axis=1)  # Drop the target column
y = data['Category']  # Target column

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert non-numeric columns to numeric using one-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)
# Train-test split (keeping 20% of the data for test sets)
X_train, X_test_temp, y_train, y_test_temp = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Split the test set into two halves
X_test_1, X_test_2, y_test_1, y_test_2 = train_test_split(X_test_temp, y_test_temp, test_size=0.5, random_state=42)


training the GBT

In [109]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the GBT model
gbt_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="mlogloss")

# Train the model using preprocessed training data
gbt_model.fit(X_train, y_train)

# Evaluate the model on both test splits
y_pred_test_1 = gbt_model.predict(X_test_1)
y_pred_test_2 = gbt_model.predict(X_test_2)

# Evaluate performance
print("GBT Model Performance:")
print("Test 1 Accuracy:", accuracy_score(y_test_1, y_pred_test_1))
print("Test 2 Accuracy:", accuracy_score(y_test_2, y_pred_test_2))
print("\nClassification Report (Test 1):")
print(classification_report(y_test_1, y_pred_test_1))
print("\nClassification Report (Test 2):")
print(classification_report(y_test_2, y_pred_test_2))


GBT Model Performance:
Test 1 Accuracy: 0.6
Test 2 Accuracy: 0.4

Classification Report (Test 1):
              precision    recall  f1-score   support

           0       0.75      0.50      0.60        12
           1       0.50      0.75      0.60         8

    accuracy                           0.60        20
   macro avg       0.62      0.62      0.60        20
weighted avg       0.65      0.60      0.60        20


Classification Report (Test 2):
              precision    recall  f1-score   support

           0       0.64      0.47      0.54        15
           1       0.11      0.20      0.14         5

    accuracy                           0.40        20
   macro avg       0.37      0.33      0.34        20
weighted avg       0.51      0.40      0.44        20



C:\Users\Satish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [21:51:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
